# Vectorized fields

Is there a way to vectorize getting and storing the fields (ie modes of various kinds)?

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad_vec
from slab import SlabExact, plot_complex, plotlogf
plt.style.use('dark_background')
%matplotlib widget

In [52]:
A = SlabExact(symmetric=True)

 Right now the method evaluate_fields is vectorized for Z.  Is it already vectorized for x and z as well?

In [53]:
A.evaluate_fields(np.array([1,2,3]), 1.1)

array([-0.61674514+0.j, -0.72155274+0.j, -0.23420546+0.j])

In [54]:
A.evaluate_fields(1, np.array([1,2,3]))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

No it isn't.  Here is a local copy of the function:

In [55]:
def evaluate_fields(C, x, z=0, Normalizer=None, field_type='TE',
                    mode_type='radiation', sign='+1', plane='Z'):
    '''Return value of field with propagation constant C at x, z in plane
    determined by 'plane'. Vectorized in C.'''

    if plane not in ['Z', 'Beta', 'Psi']:
        raise ValueError("Plane must be 'Beta', 'Z' or 'Psi'.")

    i = A.region_index(x)

    if plane == 'Beta':
        Beta = C
        Zi = np.array(A.Zi_from_Beta(C, ni=A.ns[i]))

    elif plane == 'Z':
        Zi = np.array(A.Zi_from_Z0(C, ni=A.ns[i]))
        Beta = A.Beta_from_Zi(Zi, ni=A.ns[i])

    else:
        raise NotImplementedError("Psi plane not yet implemented.")

    Cs = A.coefficients(C, Normalizer=Normalizer,
                           up_to_region=i, mode_type=mode_type,
                           field_type=field_type, sign=sign, plane=plane)

    return (Cs[..., 0, i] * np.exp(1j * Zi * x) +
            Cs[..., 1, i] * np.exp(-1j * Zi * x)) * np.exp(1j * Beta * z)

There are a few things that need to be done to vectorize it:

- Region indexes needs to be vectorized for x.

- Getting Zs on each region needs to be vectorized for x.
  
- Coefficient function itself needs to be vectorized (for x,z).  This will involve using a higher dimensional index (j in function above).


Below is a vectorized version of getting region indices:

In [56]:
A.region_index(np.array([-1,1,8])), A.ns_from_xs(np.array([-1,1,3]))

(array([0, 1, 2]), array([1.3, 1.5, 1.3]))

In [57]:
def region_indices(xs):
    '''Return region indices in which (non-dimensional) xs lie.  Indexing
    is based on left-continuity, so if x is in (rho_i, rho_i+1) it gets
    index i.  Points outside of domain on left get index 0, on right get
    index len(Rhos)-1.'''
    try:
        len(xs)
        xs = np.array(xs)
    except TypeError:
        xs = np.array([xs])
    idx = np.zeros_like(xs, dtype=int)

    idx[np.where(xs <= A.Rhos[0])] = 0
    for j in range(len(A.Rhos)-1):
        idx[np.where((xs <= A.Rhos[j+1]) * (xs > A.Rhos[j]))] = j
    idx[np.where((xs > A.Rhos[-1]))] = j

    return idx

def ns_from_xs(xs):
    '''Return refractive indices from region in which (non-dimensional) xs
    lie.'''
    return A.ns[region_indices(xs)]

In [79]:
xs = np.array([-4,1, .4, 3])

In [80]:
region_indices(xs)

array([0, 1, 1, 2])

In [81]:
ns_from_xs(xs)

array([1.3, 1.5, 1.5, 1.3])

In [82]:
A.ns_from_xs(xs)

array([1.3, 1.5, 1.5, 1.3])

Okay, now that we have x-vectorized getting the refractive indices, we need to vectorize getting the Z values:

In [83]:
def Beta_from_Z(Zs, n=None):
    """Find scaled Beta from scaled Z."""
    if n is None:
        n = self.n0
    return np.sqrt(self.K0**2 * self.n0**2 - Zs**2, dtype=complex)

def Zi_from_Z0(Z0, ni):
    '''Get Zi's on regions i (with refractive indices ni) from Z0s.
    Vectorized for Z0s and ni'''
    # try:
    #     len(ni)
    #     ni = np.array(ni)
    # except TypeError:
    #     ni = np.array([ni])

    Z0, ni = np.array(Z0), np.array(ni)
    return np.sqrt((ni[np.newaxis]**2 - A.n0**2) * A.K0**2 +
                   Z0[..., np.newaxis]**2, dtype=complex)
    
def Zi_from_Z0_og(Z0, ni):
    '''Get Z on region i (with refractive index ni) from Z0'''
    Z0, ni = np.array(Z0), np.array(ni)
    return np.sqrt((ni[np.newaxis]**2 - A.n0**2) * A.K0**2 +
                   Z0[..., np.newaxis]**2, dtype=complex)[..., 0]


In [86]:
Zi_from_Z0([1,2], 2).shape

(2, 1)

Oh well only difference really is that we just pick the 0 column for the og version.  This gets at the need to vectorize coefficients.

Okay working on this now:

In [97]:
def evaluate_fields_Z(Zs, xs, zs=0, field_type='TE',
                      mode_type='radiation', sign='+1',
                      paper_method=False):
    '''Return value of field with propagation constant Zj(Z) at x, z=0.'''
    ns = ns_from_xs(xs)
    Zj = Zi_from_Z0(Zs, ns)
    # Betas = A.Beta_from_Zi(Zs, ni=A.n0)
    js = region_indices(xs)
    jmax = np.max(js)
    Cs = []
    for i in range(jmax):
        Cs.append(A.coefficients(Zs, up_to_region=i, mode_type=mode_type,
                                 field_type=field_type, sign=sign))
    return (np.exp(1j * Zj * xs) * Cs[..., 0, j] + np.exp(-1j * Zj * xs) * Cs[..., 1, j])
    # return Betas[..., np.newaxis] * zs[np.newaxis]

In [98]:
evaluate_fields_Z(np.array([[1,2],[3,4]]), np.array([-1,2,3]), zs=np.array([3]))

NameError: name 'j' is not defined

But we still need to further vectorize the coefficients. Below is a simpler version of that function to work with:

In [17]:
def coefficients_Z(self, Z, c0=1, field_type='TE',
                   mode_type='radiation', sign='+1',
                   up_to_region=-1, rounding=12):
    """Return field coefficients given propagation constant Beta."""

    if field_type != 'TE' and field_type != 'TM':
        raise ValueError('Must have field_type either TE or TM.')

    # Single scalar inputs need to be given at least a single dimension
    Z = np.array(Z, dtype=complex)

    # Set up initial vector array M0.
    # This will contain initial vector for each coefficient array,
    # and also be overwritten to store the new coefficients
    # as we apply transfer matrix.
    M0 = np.zeros(Z.shape + (2, 1), dtype=complex)

    if mode_type == 'guided':
        check_idx = 1
        M0[..., :, 0] = np.array([0, c0])

    elif mode_type == 'leaky':
        check_idx = 0
        M0[..., :, 0] = np.array([c0, 0])

    elif mode_type == 'radiation':

        if len(self.ns) > 1:

            M = self.transmission_matrix_Z(Z, field_type=field_type)
            detM = self.transmission_determinant_Z(Z, field_type=field_type)

            frac = (-M[..., 1, 0] * detM) / M[..., 0, 1]

            b = int(sign) * np.sqrt(frac, dtype=complex)

            factor = M[..., 0, 0].conj() + b - M[..., 0, 1].conj()

            C1 = (b.conj() - M[..., 0, 1]) * factor
            C2 = M[..., 0, 0] * factor
            M0[..., 0, :] = C1[..., np.newaxis]
            M0[..., 1, :] = C2[..., np.newaxis]
            factor2 = np.sqrt(4 * M0[..., 0, :] * M0[..., 1, :],
                              dtype=complex)[..., np.newaxis]
            M0 *= np.sqrt(2/np.pi) * 1 / factor2

        else:
            # Single layer case.
            if int(sign) == 1:
                phase = 0
            else:
                phase = np.pi/2
            phase_term = np.exp(1j*phase)
            C = c0 * phase_term
            M0[..., :, 0] = np.array([C, C.conjugate()], dtype=complex).T

    else:
        raise ValueError('Mode type must be guided, leaky or radiation.')

    Rhos = self.Rhos
    ns = self.ns

    if up_to_region >= 0:
        up_to_region = up_to_region - len(Rhos) + 1

    Coeffs = np.zeros(Z.shape + (2, len(Rhos)+up_to_region),
                      dtype=complex)

    # set first vectors in each coefficient array
    Coeffs[..., :, 0] = M0[..., :, 0]

    for i in range(1, len(Rhos)+up_to_region):
        nl, nr = ns[i-1], ns[i]
        Rho = Rhos[i]
        T = self.transfer_matrix_Z(Z, Rho, nl, nr,
                                   field_type=field_type)

        M0 = (T @ M0)  # apply T to vectors
        Coeffs[..., :, i] = M0[..., :, 0]  # update coefficient arrays

    if len(Z) == 1:
        Coeffs = Coeffs[0]
    # Round to avoid false blowup, noted in guided modes.
    # Fundamental had lower error and rounding=16 worked, but HOMs
    # had more noise and required rounding=12
    Coeffs = np.round(Coeffs, decimals=rounding)

    return Coeffs

In [18]:
coefficients_Z(A, 1)

AttributeError: 'SlabExact' object has no attribute 'transmission_matrix_Z'